# Kaggle Dataset Download to Google Drive

このノートブックは、Kaggleのデータセットを直接Google Driveにダウンロードします。

## 使い方
1. Google Colabでこのノートブックを開く
2. `COMPETITION`変数を自分のコンペ名に変更
3. セルを順番に実行
4. kaggle.jsonファイルをアップロード
5. データがGoogle Driveに保存される

## 事前準備
1. https://www.kaggle.com/settings → API → Create New Token
2. `kaggle.json` ファイルがダウンロードされる
3. このファイルを安全な場所に保存しておく

## メリット
- ローカルディスク容量を節約
- Colabの高速ネットワークでダウンロード
- 大容量データセットでも問題なし

In [ ]:
# ========================================
# Configuration
# ========================================

# コンペ名を設定（必須）
COMPETITION = "titanic"  # 例: "titanic", "house-prices-advanced-regression-techniques"

# Google Driveのベースパス（通常は変更不要）
DRIVE_BASE = "/content/drive/MyDrive/Kaggle"

In [ ]:
# ========================================
# 1. Google Driveをマウント
# ========================================

from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive mounted successfully!")

In [ ]:
# ========================================
# 2. ディレクトリ構造を作成
# ========================================

import os

DRIVE_PATH = f"{DRIVE_BASE}/{COMPETITION}"

# ディレクトリ構造
directories = [
    f"{DRIVE_PATH}/data",
    f"{DRIVE_PATH}/outputs/reports",
    f"{DRIVE_PATH}/outputs/plots",
    f"{DRIVE_PATH}/outputs/models",
    f"{DRIVE_PATH}/submissions",
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"✓ Created: {directory}")

print(f"\n✓ Directory structure created in: {DRIVE_PATH}")

In [ ]:
# ========================================
# 3. Kaggle API認証設定
# ========================================

import os
from google.colab import files

# kaggle.jsonをアップロード
# (https://www.kaggle.com/settings -> API -> Create New Token で取得)
print("kaggle.json ファイルをアップロードしてください")
print("(Settings -> API -> Create New Token でダウンロード)\n")

uploaded = files.upload()

# kaggle.jsonを配置
os.makedirs('/root/.kaggle', exist_ok=True)

# アップロードされたファイルを保存
for filename, content in uploaded.items():
    with open('/root/.kaggle/kaggle.json', 'wb') as f:
        f.write(content)
    print(f"\n✓ {filename} を読み込みました")

os.chmod('/root/.kaggle/kaggle.json', 0o600)

print("✓ Kaggle API configured successfully!")

In [ ]:
# ========================================
# 4. Kaggle CLIをインストール
# ========================================

# 依存関係の問題を回避するためバージョン指定
!pip uninstall -y kaggle kagglesdk -q 2>/dev/null
!pip install -q kaggle==1.6.14

# インストール確認
!kaggle --version

print("\n✓ Kaggle CLI installed successfully!")

In [ ]:
# ========================================
# 5. データセットをダウンロード
# ========================================

import os

data_dir = f"{DRIVE_PATH}/data"
os.chdir(data_dir)

print(f"Downloading dataset for competition: {COMPETITION}")
print(f"Target directory: {data_dir}\n")

# コンペデータをダウンロード
!kaggle competitions download -c {COMPETITION}

print("\n✓ Download completed!")

In [ ]:
# ========================================
# 6. ZIPファイルを解凍
# ========================================

import os
import glob

data_dir = f"{DRIVE_PATH}/data"
os.chdir(data_dir)

# すべてのZIPファイルを解凍
zip_files = glob.glob("*.zip")

if zip_files:
    print(f"Found {len(zip_files)} ZIP file(s):\n")
    for zip_file in zip_files:
        print(f"Extracting: {zip_file}")
        !unzip -q {zip_file}
        !rm {zip_file}
        print(f"✓ Extracted and removed: {zip_file}")
else:
    print("No ZIP files found. Data may already be extracted.")

print(f"\n✓ Extraction completed!")
print(f"\nFiles in {data_dir}:")
!ls -lh

In [ ]:
# ========================================
# 7. データの確認
# ========================================

import pandas as pd
import os

data_dir = f"{DRIVE_PATH}/data"
files = os.listdir(data_dir)

print(f"Downloaded files in {data_dir}:")
print(f"{'='*60}")
for file in sorted(files):
    file_path = os.path.join(data_dir, file)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"  - {file} ({size_mb:.2f} MB)")

print(f"\n{'='*60}")
print("Sample data preview:")
print(f"{'='*60}\n")

# CSVファイルのサンプル表示
csv_files = [f for f in files if f.endswith('.csv')]
for csv_file in csv_files:
    try:
        df = pd.read_csv(f"{data_dir}/{csv_file}")
        print(f"📊 {csv_file}")
        print(f"   Shape: {df.shape}")
        print(f"   Columns: {list(df.columns)}")
        print(f"\n   First 3 rows:")
        print(df.head(3).to_string(index=False))
        print(f"\n{'='*60}\n")
    except Exception as e:
        print(f"   Error reading {csv_file}: {e}\n")

print("\n✓ Setup completed successfully!")
print(f"\nData is now available at: {data_dir}")
print("\nNext steps:")
print("1. If you have Google Drive Desktop, the data will sync to your local machine")
print("2. Start developing your analysis code with Claude Code locally")
print("3. Use other Colab notebooks to run training with GPU/TPU")

---

## トラブルシューティング

### エラー: "401 - Unauthorized"
- APIトークンが無効または期限切れの可能性があります
- https://www.kaggle.com/settings → API → Create New Token で新しいトークンを生成してください
- **重要**: 新しいトークン形式（KGAT_で始まる）は `kaggle>=1.6.14` が必要です
- ランタイムを再起動してから再度実行してください

### エラー: "403 - Forbidden"
- コンペの規約に同意していない可能性があります
- Kaggleのコンペページで "Join Competition" または "Late Submission" をクリックして規約に同意してください

### エラー: "404 - Not found"
- コンペ名が間違っている可能性があります
- `COMPETITION`変数を正しいコンペ名に設定してください
- コンペ名はKaggle URLから取得できます: `https://www.kaggle.com/competitions/{competition-name}`

### エラー: "ImportError: cannot import name ... from 'kagglesdk'"
- kaggleパッケージの依存関係問題です
- ランタイムを再起動し、`pip install kaggle==1.6.14` を最初に実行してください

### ダウンロードが遅い
- Colabのネットワークは通常高速ですが、大容量データセット（10GB以上）の場合は時間がかかります
- セルの実行が完了するまで待ちましょう

### Google Driveの容量不足
- Google Driveの容量を確認してください（無料版は15GB）
- 不要なファイルを削除するか、Google Oneで容量をアップグレードしてください

---

🤖 Generated with [Claude Code](https://claude.com/claude-code)